# Coincident Peak Probability via Monte Carlo — IESO (RTO 1‑day‑ahead)

Implementing the **Monte Carlo + Adaptive Threshold** Reproduction Pipeline of Carmona et al. (2024)

1. Make residuals: For each hour h, take the historical actual−forecast.

2. Edge trimming: Fit the tail of GPD (generalized Pareto) individually every hour, map the residuals to [0,1] using probability integral transform (PIT), and then use the standard normal quantile function Q^-1 to turn it into the "Gaussianized residual" of standard normal N(0,1).

3. Graphical model: Splice the 24-hour "Gaussian residual" into a 24-dimensional vector, assuming its joint distribution is multivariate normal, use Graphical LASSO (maximum likelihood with regularity) to estimate the accuracy matrix/covariance to get the sparse dependence structure between hours.

4. Monte Carlo: From the 24-dimensional Gauss learned, took Gauss residuals → used the previous inverse transformation to restore to the residuals of the original dimension → added back to the previous curve to get the complete set of scenes of "Tomorrow 24 hours."

5. Calculate the probability:

1CP Probability: The "Tomorrow's Sun Peak" in the scene exceeds the "running to yesterday's largest peak in the year".

Top-5 Update probability: The ratio exceeds the "run to yesterday's previous k threshold."

4CP probability: In June–9, the "maximum value of the current month to date" is used as the benchmark, and the proportion exceeding it in the scenario.

Then, it is combined with a simple threshold (such as 0.5) or "red, orange, yellow and green" grading to serve as an alarm.

## 0. Environment Setup

In [1]:
# ==== Cell 0: Environment ====
import os, warnings, math, datetime as dt
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, Optional, Tuple, List
from scipy import stats
from sklearn.covariance import GraphicalLassoCV
from sklearn.metrics import precision_recall_fscore_support, brier_score_loss, roc_auc_score
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 120)
np.random.seed(42)

try:
    from statsmodels.distributions.empirical_distribution import ECDF
except Exception:
    class ECDF:
        def __init__(self, x):
            x = np.asarray(x)
            x = x[np.isfinite(x)]
            self.x = np.sort(x)
            self.n = len(self.x)

        def __call__(self, v):
            v = np.asarray(v)
            return np.searchsorted(self.x, v, side="right") / max(self.n, 1)

print("pandas:", pd.__version__)


pandas: 2.2.3



## 1. Data Reading (IESO)
- File: `/mnt/data/IESO-Ontario Demand 2022-2025 1 day ahead.csv`
- Columns: `ECA ... Forecast`, `RTO ... Forecast`, `TESLA ... Forecast`, `TESLA ... Actual`
- The default selection of `RTO`


In [2]:
# ==== Cell 1: Parameters ====
PATH = "IESO-Ontario Demand 2022-2025 1 day ahead.csv"

FORECAST_SOURCES = ["ECA", "RTO", "TESLA"]

# Monte Carlo Sample number and window
LAST_DAYS = 730  # None = full amount; it is recommended to zoom in the first 365~730
N_SIMS = 2000  # 600 starts, 2000+ is more stable

BUDGET_1CP = 6
BUDGET_TOP5 = 12
BUDGET_4CP_SUMMER = 8  # Total summer budget (June-September)

SAVE_OUTPUTS = True
OUT_DIR = "./output_cp_prob_mc_ieso"
os.makedirs(OUT_DIR, exist_ok=True)

In [3]:
# ==== Cell 2: Load CSV & detect columns ====
assert os.path.exists(PATH), f"Data file not found: {PATH}"
raw = pd.read_csv(PATH)

# Automatically identify columns
all_cols = raw.columns.tolist()
FCAST_COLS = [c for c in all_cols if "Historic Forecast" in c]
ACTUAL_CANDIDATES = [c for c in all_cols if "Actual" in c]

assert len(FCAST_COLS) >= 1, "The *Historic Forecast* column was not found, please check the file."
assert len(ACTUAL_CANDIDATES) >= 1, "The *Actual* column was not found, please check the file."

ACTUAL_COL = ACTUAL_CANDIDATES[0]

print("Forecast columns:", FCAST_COLS)
print("Actual column:", ACTUAL_COL)
print(raw.head(3))


Forecast columns: ['ECA: IESO-Ontario Demand Historic Forecast', 'RTO: IESO-Ontario Demand Historic Forecast', 'TESLA: IESO-Ontario Demand Historic Forecast']
Actual column: TESLA: IESO-Ontario Demand Actual
        Date  Time  ECA: IESO-Ontario Demand Historic Forecast  \
0  1/01/2022  1:00                                     13589.5   
1  1/01/2022  2:00                                     13104.1   
2  1/01/2022  3:00                                     12681.4   

   RTO: IESO-Ontario Demand Historic Forecast  \
0                                     13521.8   
1                                     13079.0   
2                                     12677.8   

   TESLA: IESO-Ontario Demand Historic Forecast  \
0                                       13521.8   
1                                       13079.0   
2                                       12677.8   

   TESLA: IESO-Ontario Demand Actual  
0                           13542.75  
1                           13253.67  
2       

In [4]:
# ==== Cell 3: Timestamp parser (handles '24:00') ====
def parse_timestamp(date_str, time_str):
    t = str(time_str).strip()
    if t.startswith("24:"):
        dt0 = dt.datetime.strptime(str(date_str).strip(), "%d/%m/%Y") + dt.timedelta(days=1)
        parts = t.split(":")
        mm = parts[1] if len(parts) >= 2 else "00"
        ss = parts[2] if len(parts) == 3 else "00"
        return pd.to_datetime(f"{dt0.strftime('%d/%m/%Y')} 00:{mm}:{ss}", dayfirst=True)
    return pd.to_datetime(f"{date_str} {time_str}", dayfirst=True, errors="coerce")


ts = [parse_timestamp(d, t) for d, t in zip(raw["Date"], raw["Time"])]
ts = pd.to_datetime(ts)
assert pd.isna(ts).sum() == 0, "The time stamp parsing exists NaT, please check the 'Date'/'Time' format."
print("Time range:", ts.min(), "→", ts.max(), "len:", len(ts))


Time range: 2022-01-01 01:00:00 → 2025-05-16 00:00:00 len: 29543


In [5]:
# ==== Cell 4 (fixed): Safe hourly regularization + base builder ====
def hourly_regularize_safe(df: pd.DataFrame) -> pd.DataFrame:
    out_parts = []
    for (src, zn), g in df.groupby(['forecast_src', 'zone'], sort=False):
        g = g.sort_values('timestamp')
        g2 = g.set_index('timestamp').asfreq('H')
        # Only fill numeric columns to avoid object column participation
        num_cols = [c for c in g2.columns if pd.api.types.is_numeric_dtype(g2[c])]
        g2[num_cols] = g2[num_cols].ffill().bfill()
        g2 = g2.reset_index()
        g2['forecast_src'] = src
        g2['zone'] = zn
        out_parts.append(g2)
    out = pd.concat(out_parts, ignore_index=True)
    out = out.sort_values('timestamp').reset_index(drop=True)
    return out


def build_base_df(forecast_col: str) -> pd.DataFrame:
    df = pd.DataFrame({
        "timestamp": ts,
        "zone": "IESO",
        "actual_mw": pd.to_numeric(raw[ACTUAL_COL], errors="coerce"),
        "da_forecast_mw": pd.to_numeric(raw[forecast_col], errors="coerce"),
        "forecast_src": forecast_col.split(":")[0].strip()
    }).dropna().sort_values("timestamp").reset_index(drop=True)

    # Merge DST callback repeat hours (take the mean of the same hour)
    df = df.groupby(['forecast_src', 'zone', 'timestamp'], as_index=False) \
        .agg({'actual_mw': 'mean', 'da_forecast_mw': 'mean'})

    df = hourly_regularize_safe(df)
    df = df.dropna(subset=['actual_mw', 'da_forecast_mw']).sort_values('timestamp').reset_index(drop=True)
    return df


_test_col = FCAST_COLS[0]
_df_test = build_base_df(_test_col)
print(_test_col, _df_test.shape)
print(_df_test[['timestamp', 'zone', 'forecast_src', 'actual_mw', 'da_forecast_mw']].head(3))


ECA: IESO-Ontario Demand Historic Forecast (29536, 5)
            timestamp  zone forecast_src  actual_mw  da_forecast_mw
0 2022-01-01 01:00:00  IESO          ECA   13542.75         13589.5
1 2022-01-01 02:00:00  IESO          ECA   13253.67         13104.1
2 2022-01-01 03:00:00  IESO          ECA   12683.29         12681.4


## 2. Construct the daily matrix and tags (1CP/Top‑5)

In [6]:
# ==== NEW (REPLACE your daily-label cell): Hourly thresholds with PJM summer reset ====
def build_hourly_features(df_base: pd.DataFrame) -> pd.DataFrame:
    x = df_base.copy()
    x["date"] = x["timestamp"].dt.floor("D")
    x["year"] = x["timestamp"].dt.year
    x["month"] = x["timestamp"].dt.month
    x["hour"] = x["timestamp"].dt.hour
    x["residual"] = x["actual_mw"] - x["da_forecast_mw"]
    return x


def make_hourly_matrix(features_hourly: pd.DataFrame) -> pd.DataFrame:
    """Turn hourly residuals into a 24-dim row per *day* for modeling."""
    mat = features_hourly.pivot_table(index="date", columns="hour",
                                      values="residual", aggfunc="mean")
    for h in range(24):
        if h not in mat.columns: mat[h] = np.nan
    mat = mat[[h for h in range(24)]].sort_index()
    mat.columns = [f"res_h{h:02d}" for h in range(24)]
    # attach year/month for each date (from any hour of that day)
    ym = features_hourly.groupby("date")[["year", "month"]].first()
    mat = mat.merge(ym, left_index=True, right_index=True, how="left")
    mat = mat.dropna(subset=[f"res_h{h:02d}" for h in range(24)]).reset_index()  # keep date col
    return mat  # columns: date, res_h00..res_h23, year, month


def compute_pjm_summer_thresholds_hourly(df_base: pd.DataFrame) -> pd.DataFrame:
    """
    For each *day*, compute thresholds using *hourly actuals* seen up to yesterday,
    within PJM summer (Jun–Sep) and reset each year on Jun 1.
    Returns: DataFrame indexed by date with columns:
      - summer_running_max_prev_hourly
      - summer_top5_thr_prev_hourly
      - is_summer_day (flag for Jun–Sep)
    """
    x = df_base.copy()
    x["date"] = x["timestamp"].dt.floor("D")
    x["year"] = x["timestamp"].dt.year
    x["month"] = x["timestamp"].dt.month
    x["is_summer"] = x["month"].isin([6, 7, 8, 9])

    # prepare container per date
    dates = np.sort(x["date"].unique())
    recs = []

    for y, g_year in x.groupby("year"):
        g_sum = g_year[g_year["is_summer"]].sort_values("timestamp")
        if g_sum.empty:
            # still return records for dates in this year with NaNs
            for d in np.sort(g_year["date"].unique()):
                recs.append({"date": d, "summer_running_max_prev_hourly": np.nan,
                             "summer_top5_thr_prev_hourly": np.nan,
                             "is_summer_day": bool(int(pd.Timestamp(d).month in [6, 7, 8, 9]))})
            continue

        # iterate summer dates of this year in order
        for d in np.sort(g_year["date"].unique()):
            is_summer_day = int(pd.Timestamp(d).month in [6, 7, 8, 9])
            if not is_summer_day:
                recs.append({"date": d, "summer_running_max_prev_hourly": np.nan,
                             "summer_top5_thr_prev_hourly": np.nan, "is_summer_day": False})
                continue

            # only use hours strictly before this date (up to yesterday)
            prior_hours = g_sum[g_sum["date"] < d]["actual_mw"].values
            if len(prior_hours) == 0:
                recs.append({"date": d, "summer_running_max_prev_hourly": np.nan,
                             "summer_top5_thr_prev_hourly": np.nan, "is_summer_day": True})
                continue

            M_prev = float(np.max(prior_hours))
            if len(prior_hours) >= 5:
                kth = np.partition(prior_hours, -5)[-5]  # 5th highest so far
            else:
                kth = np.nan

            recs.append({"date": d,
                         "summer_running_max_prev_hourly": M_prev,
                         "summer_top5_thr_prev_hourly": float(kth),
                         "is_summer_day": True})

    thr = pd.DataFrame(recs).set_index("date").sort_index()
    return thr


## 3. Edge Gauss + Graphical LASSO (Related within the day)

In [7]:
# ==== Cell 6 (REPLACE): Paper-consistent upper-tail POT-GPD marginals ====
from dataclasses import dataclass
from typing import Dict
import numpy as np
import pandas as pd
from scipy.stats import genpareto, norm


@dataclass
class MarginalPOT:
    hour: int
    p_u: float  # upper-tail threshold quantile (e.g., 0.90)
    u: float  # threshold value at p_u
    xi: float  # GPD shape
    beta: float  # GPD scale ( >0 )
    center_vals: np.ndarray  # sorted residuals in (-inf, u], used as ECDF in body

    # piecewise CDF: body via ECDF, upper tail via GPD (POT)
    def cdf(self, x: np.ndarray) -> np.ndarray:
        x = np.asarray(x, dtype=float)
        u = np.empty_like(x)

        # body: x <= u  --> map by ECDF to [0, p_u]
        mask_body = x <= self.u
        if mask_body.any():
            vals = self.center_vals
            ranks = np.searchsorted(vals, x[mask_body], side="right")
            h = np.clip(ranks / max(len(vals), 1), 0.0, 1.0)  # ECDF in [0,1]
            u[mask_body] = self.p_u * h  # scale to [0, p_u]

        # upper tail: x > u  --> POT: F(x) = p_u + (1-p_u) * GPD(x-u)
        mask_tail = ~mask_body
        if mask_tail.any():
            y = np.maximum(x[mask_tail] - self.u, 0.0)
            # GPD CDF on exceedances y
            if self.beta <= 0:
                g = 1.0 - np.exp(-y / (np.std(self.center_vals) + 1e-6))  # fallback
            else:
                g = genpareto.cdf(y, c=self.xi, loc=0.0, scale=self.beta)
            u[mask_tail] = self.p_u + (1.0 - self.p_u) * g

        return np.clip(u, 1e-9, 1 - 1e-9)

    # piecewise PPF: invert cdf()
    def ppf(self, u: np.ndarray) -> np.ndarray:
        u = np.asarray(u, dtype=float)
        x = np.empty_like(u)

        # body: 0 <= u <= p_u  --> inverse-ECDF on center_vals
        mask_body = u <= self.p_u
        if mask_body.any():
            vals = self.center_vals
            q = np.where(self.p_u > 0, u[mask_body] / self.p_u, 0.0)  # [0,1]
            idx = np.clip((q * (len(vals) - 1)).astype(int), 0, max(len(vals) - 1, 0))
            x[mask_body] = vals[idx]

        # tail: u > p_u  --> u = p_u + (1-p_u)*GPD(y) => GPD(y) = (u-p_u)/(1-p_u)
        mask_tail = ~mask_body
        if mask_tail.any():
            q = np.clip((u[mask_tail] - self.p_u) / (1.0 - self.p_u), 0.0, 1.0)
            if self.beta <= 0:
                lam = 1.0 / (np.std(self.center_vals) + 1e-6)
                y = -np.log(np.clip(1.0 - q, 1e-12, 1.0)) / lam
            else:
                y = genpareto.ppf(q, c=self.xi, loc=0.0, scale=self.beta)
            x[mask_tail] = self.u + np.maximum(y, 0.0)

        return x

    # wrappers used downstream
    def to_normal(self, x: np.ndarray) -> np.ndarray:
        return norm.ppf(self.cdf(x))

    def from_normal(self, z: np.ndarray) -> np.ndarray:
        return self.ppf(norm.cdf(z))


def fit_marginals_pot(features: pd.DataFrame,
                      p_u: float = 0.90,
                      min_tail_n: int = 50) -> Dict[int, MarginalPOT]:
    """
    Paper-consistent POT-GPD on the *upper tail* per hour.
    Body (<= u) uses ECDF to map to [0, p_u]; tail (> u) uses GPD to map to (p_u, 1].
    """
    out: Dict[int, MarginalPOT] = {}
    for h in range(24):
        vals = features[f"res_h{h:02d}"].values.astype(float)
        vals = vals[np.isfinite(vals)]
        vals_sorted = np.sort(vals)

        # threshold at p_u (e.g., 90% quantile)
        u = float(np.quantile(vals_sorted, p_u))
        body = vals_sorted[vals_sorted <= u]
        exc = vals_sorted[vals_sorted > u] - u

        # GPD fit (exceedances)
        if len(exc) >= min_tail_n:
            try:
                xi, loc, beta = genpareto.fit(exc, floc=0.0)
                beta = float(max(beta, 1e-9))
            except Exception:
                xi, beta = 0.0, float(np.std(exc) + 1e-6)
        else:
            # tail too short -> robust fallback
            xi, beta = 0.0, float(np.std(exc) + 1e-6) if len(exc) else 0.0

        out[h] = MarginalPOT(
            hour=h, p_u=p_u, u=u,
            xi=float(xi), beta=float(beta),
            center_vals=body if len(body) else vals_sorted
        )
    return out


def gaussianize(features: pd.DataFrame, transforms: Dict[int, MarginalPOT]) -> np.ndarray:
    Z = np.zeros((len(features), 24))
    it = features.itertuples(index=False)
    for i, row in enumerate(it):
        for h in range(24):
            Z[i, h] = transforms[h].to_normal(np.array([getattr(row, f"res_h{h:02d}")]))[0]
    return Z

## 4. Scene generation and CP probability

In [8]:
# ==== Cell 7: Scenario generation + probabilities ====
def build_daily_forecast_matrix(df: pd.DataFrame) -> pd.DataFrame:
    x = df.copy()
    x["date"] = x["timestamp"].dt.floor("D")
    x["hour"] = x["timestamp"].dt.hour
    mat = x.pivot_table(index="date", columns="hour", values="da_forecast_mw", aggfunc="mean")
    for h in range(24):
        if h not in mat.columns:
            mat[h] = np.nan
    mat = mat[[h for h in range(24)]].sort_index()
    mat.columns = [f"fh{h:02d}" for h in range(24)]
    return mat


def scenario_probabilities_pjm_hourly(df_base: pd.DataFrame,
                                      feats_daily24: pd.DataFrame,
                                      transforms: Dict[int, MarginalPOT],
                                      cov: np.ndarray,
                                      n_sims: int = 600,
                                      last_days: int | None = 365,
                                      return_peak_hour: bool = True) -> pd.DataFrame:
    # 24h DA matrix per date
    x = df_base.copy()
    x["date"] = x["timestamp"].dt.floor("D")
    x["hour"] = x["timestamp"].dt.hour
    fmat = x.pivot_table(index="date", columns="hour", values="da_forecast_mw", aggfunc="mean")
    for h in range(24):
        if h not in fmat.columns: fmat[h] = np.nan
    fmat = fmat[[h for h in range(24)]].sort_index()
    fmat.columns = [f"fh{h:02d}" for h in range(24)]

    # thresholds with PJM summer reset
    thr = compute_pjm_summer_thresholds_hourly(df_base)

    # optional evaluation window
    if last_days is not None and len(fmat.index) > 0:
        cutoff = fmat.index.max() - pd.Timedelta(days=last_days)
        fmat = fmat[fmat.index > cutoff]

    # MC samples shared across dates
    rng = np.random.default_rng(9)
    z_all = rng.multivariate_normal(mean=np.zeros(24), cov=cov, size=n_sims)
    e_all = np.vstack([transforms[h].from_normal(z_all[:, h]) for h in range(24)]).T

    out = []
    for date, row in fmat.iterrows():
        if date not in thr.index or not bool(thr.loc[date, "is_summer_day"]):
            continue
        fvec = row.values.astype(float)
        if not np.isfinite(fvec).all():
            continue

        sims = fvec.reshape(1, -1) + e_all  # (S,24)
        sim_max = sims.max(axis=1)
        M_prev = thr.loc[date, "summer_running_max_prev_hourly"]
        T5_prev = thr.loc[date, "summer_top5_thr_prev_hourly"]
        p1 = float((sim_max > M_prev).mean()) if np.isfinite(M_prev) else np.nan
        p5 = float((sim_max > T5_prev).mean()) if np.isfinite(T5_prev) else np.nan

        rec = {"date": date,
               "p_1cp_hourly_summer": p1,
               "p_top5_hourly_summer": p5}

        if return_peak_hour:
            peak_idx = np.argmax(sims, axis=1)
            ph = np.bincount(peak_idx, minlength=24) / sims.shape[0]
            rec.update({f"ph_{h:02d}": float(ph[h]) for h in range(24)})
            rec["peak_hour_mlh"] = int(np.argmax(ph))  # most likely hour (0-23)

        out.append(rec)

    prob = pd.DataFrame(out).set_index("date").sort_index()
    return prob

In [9]:
# ==== NEW: 4CP (Monthly Top-4) thresholds & probabilities for summer (Jun–Sep) ====
import numpy as np
import pandas as pd
from typing import Optional, Dict


def compute_summer_monthly_topk_thresholds_hourly(df_base: pd.DataFrame,
                                                  k: int = 4) -> pd.DataFrame:
    """
    对每个夏季月份（6–9 月），构造“到昨日为止，本月第 k 高 *小时*负荷”的运行阈值。
    返回：index=date；列：
      - m4_hourly_prev (float)   本月 Top-k 的第 k 名阈值（小时口径）
      - is_summer_month_day (bool) 本日是否属于夏季月份
      - year, month
    """
    x = df_base.copy()
    x["date"] = x["timestamp"].dt.floor("D")
    x["year"] = x["timestamp"].dt.year
    x["month"] = x["timestamp"].dt.month

    recs = []
    for (y, m), g in x.groupby(["year", "month"]):
        is_summer = (m in [6, 7, 8, 9])
        dates = np.sort(g["date"].unique())
        # 按本月逐日滚动
        for d in dates:
            if not is_summer:
                recs.append({"date": d, "year": y, "month": m,
                             "m4_hourly_prev": np.nan, "is_summer_month_day": False})
                continue
            prior = g[g["date"] < d]["actual_mw"].values  # 本月到昨日为止的全部小时实测
            if len(prior) >= k:
                kth = np.partition(prior, -k)[-k]
            else:
                kth = np.nan
            recs.append({"date": d, "year": y, "month": m,
                         "m4_hourly_prev": float(kth), "is_summer_month_day": True})

    thr4h = pd.DataFrame(recs).set_index("date").sort_index()
    return thr4h


def compute_summer_monthly_topk_thresholds_daily(df_base: pd.DataFrame,
                                                 k: int = 4) -> pd.DataFrame:
    """
    对每个夏季月份（6–9 月），构造“到昨日为止，本月第 k 高 *日峰*”的运行阈值。
    返回：index=date；列：
      - m4_daily_prev (float)    本月 Top-k 的第 k 名阈值（日口径：以每日最高小时作为日峰）
      - is_summer_month_day (bool) 本日是否属于夏季月份
      - year, month
    """
    x = df_base.copy()
    x["date"] = x["timestamp"].dt.floor("D")
    x["year"] = x["timestamp"].dt.year
    x["month"] = x["timestamp"].dt.month
    # 每日“日峰”
    day_max = x.groupby(["year", "month", "date"])["actual_mw"].max().rename("day_max").reset_index()

    recs = []
    for (y, m), g in day_max.groupby(["year", "month"]):
        is_summer = (m in [6, 7, 8, 9])
        dates = np.sort(g["date"].values)
        for d in dates:
            if not is_summer:
                recs.append({"date": d, "year": y, "month": m,
                             "m4_daily_prev": np.nan, "is_summer_month_day": False})
                continue
            prior = g[g["date"] < d]["day_max"].values  # 本月到昨日为止的“日峰”序列
            if len(prior) >= k:
                kth = np.partition(prior, -k)[-k]
            else:
                kth = np.nan
            recs.append({"date": d, "year": y, "month": m,
                         "m4_daily_prev": float(kth), "is_summer_month_day": True})

    thr4d = pd.DataFrame(recs).set_index("date").sort_index()
    return thr4d


def scenario_probabilities_pjm_4cp(df_base: pd.DataFrame,
                                   feats_daily24: pd.DataFrame,
                                   transforms,
                                   cov: np.ndarray,
                                   n_sims: int = 600,
                                   last_days: Optional[int] = 365,
                                   return_peak_hour: bool = False) -> pd.DataFrame:
    """
    同一天内同时计算两种 4CP 概率（6–9 月每月 Top-4）：
      - 小时口径：p_4cp_hourly_monthlyTop4  = P(明日任一小时 > 本月到昨日为止的第4高“小时”负荷阈值)
      - 日口径  ：p_4cp_daily_monthlyTop4   = P(明日日峰   > 本月到昨日为止的第4高“日峰”   阈值)
    若 return_peak_hour=True，则附带“最可能峰小时”分布（与小时口径同样的峰时统计）。
    """
    # 24h DA 矩阵
    x = df_base.copy()
    x["date"] = x["timestamp"].dt.floor("D")
    x["hour"] = x["timestamp"].dt.hour
    fmat = x.pivot_table(index="date", columns="hour", values="da_forecast_mw", aggfunc="mean")
    for h in range(24):
        if h not in fmat.columns: fmat[h] = np.nan
    fmat = fmat[[h for h in range(24)]].sort_index()
    fmat.columns = [f"fh{h:02d}" for h in range(24)]

    # 4CP 两类阈值（按月 Top-4）
    thr4h = compute_summer_monthly_topk_thresholds_hourly(df_base, k=4)
    thr4d = compute_summer_monthly_topk_thresholds_daily(df_base, k=4)

    # 可选：仅评最近 N 天
    if last_days is not None and len(fmat.index) > 0:
        cutoff = fmat.index.max() - pd.Timedelta(days=last_days)
        fmat = fmat[fmat.index > cutoff]

    # 共享 MC 样本
    rng = np.random.default_rng(19)
    z_all = rng.multivariate_normal(mean=np.zeros(24), cov=cov, size=n_sims)
    e_all = np.vstack([transforms[h].from_normal(z_all[:, h]) for h in range(24)]).T  # (S,24)

    out = []
    for date, row in fmat.iterrows():
        # 仅在夏季月份计算
        if (date not in thr4h.index) or (date not in thr4d.index) or (not bool(thr4h.loc[date, "is_summer_month_day"])):
            continue
        fvec = row.values.astype(float)
        if not np.isfinite(fvec).all():
            continue

        sims = fvec.reshape(1, -1) + e_all  # (S,24)
        sim_hourly_max = sims.max(axis=1)  # “该日任一小时”的最大值（也即当日“日峰”）
        # 阈值
        H4_prev = thr4h.loc[date, "m4_hourly_prev"]
        D4_prev = thr4d.loc[date, "m4_daily_prev"]
        p4h = float((sim_hourly_max > H4_prev).mean()) if np.isfinite(H4_prev) else np.nan
        p4d = float((sim_hourly_max > D4_prev).mean()) if np.isfinite(D4_prev) else np.nan

        rec = {"date": date,
               "p_4cp_hourly_monthlyTop4": p4h,
               "p_4cp_daily_monthlyTop4": p4d}
        if return_peak_hour:
            peak_idx = np.argmax(sims, axis=1)
            ph = np.bincount(peak_idx, minlength=24) / sims.shape[0]
            rec.update({f"ph_{h:02d}": float(ph[h]) for h in range(24)})
            rec["peak_hour_mlh"] = int(np.argmax(ph))
        out.append(rec)

    prob4 = pd.DataFrame(out).set_index("date").sort_index()
    return prob4


## 5. Adaptive Threshold + Metrics

In [10]:
# ==== Cell 8: Adaptive threshold + evaluation ====
def adaptive_threshold_series(prob: pd.Series, budget_per_year: int = 10) -> pd.Series:
    # Given quantile thresholds based on budget, the warm-up of the first 15 days is NaN
    q = 1.0 - (budget_per_year / 365.0)
    thresholds, hist = [], []
    for v in prob.values:
        if len(hist) < 15:
            thresholds.append(np.nan)
        else:
            thresholds.append(np.quantile(np.array(hist), q))
        hist.append(v if np.isfinite(v) else 0.0)
    return pd.Series(thresholds, index=prob.index)


def evaluate(prob_df: pd.DataFrame, labs: pd.DataFrame,
             target_col_prob: str, target_event_col: str, budget_per_year: int,
             restrict_summer_4cp: bool = False) -> dict:
    labs_idx = labs.set_index("date").sort_index()
    idx = prob_df.index

    if restrict_summer_4cp:
        mask = labs_idx.loc[idx, "is_4cp_month"] == 1
        idx = idx[mask.values]
        if len(idx) == 0:
            return {"brier": np.nan, "precision": np.nan, "recall": np.nan, "f1": np.nan, "auc": np.nan}

    y = labs_idx.loc[idx, target_event_col].astype(int).values
    p = prob_df.loc[idx, target_col_prob].values
    p = np.nan_to_num(p, nan=0.0)

    brier = brier_score_loss(y, p)
    thr = adaptive_threshold_series(pd.Series(p, index=idx), budget_per_year)
    alerts = (pd.Series(p, index=idx) >= thr).astype(int).values

    prec, rec, f1, _ = precision_recall_fscore_support(y, alerts, average='binary', zero_division=0)
    try:
        auc = roc_auc_score(y, p)
    except Exception:
        auc = np.nan

    return {"brier": float(brier), "precision": float(prec), "recall": float(rec),
            "f1": float(f1), "auc": float(auc)}


In [11]:
# ==== NEW: Alert layer aligned with the paper's examples ====
def fixed_threshold_signal(p: pd.Series, thr: float = 0.5) -> pd.Series:
    """Binary alert: 1 if p >= thr else 0 (default thr=0.5)."""
    return (p >= thr).astype(int)


def traffic_light_band(p: pd.Series) -> pd.Series:
    """
    RED    : p >= 0.80
    ORANGE : 0.60 <= p < 0.80
    YELLOW : 0.40 <= p < 0.60
    GREEN  : p < 0.40
    """
    cats = pd.Series(index=p.index, dtype="object")
    cats[p >= 0.80] = "RED"
    cats[(p >= 0.60) & (p < 0.80)] = "ORANGE"
    cats[(p >= 0.40) & (p < 0.60)] = "YELLOW"
    cats[p < 0.40] = "GREEN"
    return cats

# example: attach signals to your probability table
# prob_hourly = scenario_probabilities_pjm_hourly(...)
# prob_hourly["sig_1cp_bin_0p5"]   = fixed_threshold_signal(prob_hourly["p_1cp_hourly_summer"], 0.5)
# prob_hourly["sig_1cp_band"]      = traffic_light_band(prob_hourly["p_1cp_hourly_summer"])
# prob_hourly["sig_top5_bin_0p5"]  = fixed_threshold_signal(prob_hourly["p_top5_hourly_summer"], 0.5)
# prob_hourly["sig_top5_band"]     = traffic_light_band(prob_hourly["p_top5_hourly_summer"])


In [12]:
# ==== Cell 9 (UPDATED): +4CP (monthly Top-4) hourly & daily ====
from sklearn.metrics import precision_recall_fscore_support, brier_score_loss, roc_auc_score


def _eval_binary(y_true, p_prob, thr=0.5):
    y_true = np.asarray(y_true, dtype=int)
    p_prob = np.asarray(p_prob, dtype=float)
    mask = np.isfinite(y_true) & np.isfinite(p_prob)
    if mask.sum() == 0:
        return dict(brier=np.nan, precision=np.nan, recall=np.nan, f1=np.nan, auc=np.nan)
    y, p = y_true[mask], p_prob[mask]
    brier = brier_score_loss(y, p)
    y_hat = (p >= thr).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(y, y_hat, average="binary", zero_division=0)
    try:
        auc = roc_auc_score(y, p)
    except Exception:
        auc = np.nan
    return dict(brier=float(brier), precision=float(prec), recall=float(rec), f1=float(f1), auc=float(auc))


summary_rows = []
probs_by_src = {}
labels_by_src = {}

for src in FORECAST_SOURCES:
    fcols = [c for c in FCAST_COLS if c.startswith(src)]
    if not fcols:
        print(f"[WARN] No forecast column for source '{src}', skip.")
        continue
    fcol = fcols[0]
    print(f"\n>>> Running source: {src} — column: {fcol}")

    # 1) 基础小时表
    df_base = build_base_df(fcol)

    # 2) 逐小时残差 -> 按日的 24 维矩阵（供边缘拟合）
    features_hourly = build_hourly_features(df_base)
    feats_daily24 = make_hourly_matrix(features_hourly)

    # 3) POT-GPD（上尾）+ Graphical LASSO
    transforms = fit_marginals_pot(feats_daily24, p_u=0.90, min_tail_n=50)
    Z = gaussianize(feats_daily24, transforms)
    gl = GraphicalLassoCV()
    gl.fit(Z)
    cov = gl.covariance_

    # 4) 1CP/Top-5（小时口径，夏季）——你已有
    prob_hourly = scenario_probabilities_pjm_hourly(
        df_base, feats_daily24, transforms, cov,
        n_sims=N_SIMS, last_days=LAST_DAYS
    )
    prob_hourly["sig_1cp_bin_0p5"] = fixed_threshold_signal(prob_hourly["p_1cp_hourly_summer"], 0.5)
    prob_hourly["sig_1cp_band"] = traffic_light_band(prob_hourly["p_1cp_hourly_summer"])
    prob_hourly["sig_top5_bin_0p5"] = fixed_threshold_signal(prob_hourly["p_top5_hourly_summer"], 0.5)
    prob_hourly["sig_top5_band"] = traffic_light_band(prob_hourly["p_top5_hourly_summer"])

    # 5) 4CP（每月 Top-4）：同时输出“小时口径 & 日口径”
    prob_4cp = scenario_probabilities_pjm_4cp(
        df_base, feats_daily24, transforms, cov,
        n_sims=N_SIMS, last_days=LAST_DAYS,
        return_peak_hour=False
    )
    prob_4cp["sig_4cp_hourly_bin_0p5"] = fixed_threshold_signal(prob_4cp["p_4cp_hourly_monthlyTop4"], 0.5)
    prob_4cp["sig_4cp_hourly_band"] = traffic_light_band(prob_4cp["p_4cp_hourly_monthlyTop4"])
    prob_4cp["sig_4cp_daily_bin_0p5"] = fixed_threshold_signal(prob_4cp["p_4cp_daily_monthlyTop4"], 0.5)
    prob_4cp["sig_4cp_daily_band"] = traffic_light_band(prob_4cp["p_4cp_daily_monthlyTop4"])

    # 合并到一个 per-source 概率表（索引 date 对齐）
    prob_all = prob_hourly.join(prob_4cp, how="outer").sort_index()

    # 6) 构造评估标签（与 Cell 9 里 1CP/Top-5 一致；新增 4CP 两类标签）
    #    先复用你已有的“夏季小时阈值”与“日峰”
    thr_summer = compute_pjm_summer_thresholds_hourly(df_base)  # 1CP/Top5 小时口径
    day_max_actual = (
        df_base.assign(date=df_base["timestamp"].dt.floor("D"))
        .groupby("date")["actual_mw"].max()
        .to_frame("day_max_actual")
    )
    labs = thr_summer.join(day_max_actual, how="left")
    labs = labs[labs["is_summer_day"].astype(bool)]

    # 1CP/Top5 标签（小时口径，夏季）
    labs_1cp = labs[np.isfinite(labs["summer_running_max_prev_hourly"])]
    labs_top5 = labs[np.isfinite(labs["summer_top5_thr_prev_hourly"])]
    y1 = (labs_1cp["day_max_actual"].values > labs_1cp["summer_running_max_prev_hourly"].values).astype(int)
    y5 = (labs_top5["day_max_actual"].values > labs_top5["summer_top5_thr_prev_hourly"].values).astype(int)
    p1 = prob_all.reindex(labs_1cp.index)["p_1cp_hourly_summer"].values
    p5 = prob_all.reindex(labs_top5.index)["p_top5_hourly_summer"].values

    # 4CP 阈值（新增）：每月 Top-4 —— 小时口径 & 日口径
    thr4h = compute_summer_monthly_topk_thresholds_hourly(df_base, k=4)
    thr4d = compute_summer_monthly_topk_thresholds_daily(df_base, k=4)
    # 小时口径 4CP：当日“任一小时”是不是超过本月到昨日为止的第4高“小时”阈值 → 等价于 day_max_actual > m4_hourly_prev
    labs_4h = thr4h.join(day_max_actual, how="left")
    labs_4h = labs_4h[labs_4h["is_summer_month_day"].astype(bool)]
    y4h = (labs_4h["day_max_actual"].values > labs_4h["m4_hourly_prev"].values).astype(int)
    p4h = prob_all.reindex(labs_4h.index)["p_4cp_hourly_monthlyTop4"].values

    # 日口径 4CP：当日“日峰”是不是超过本月到昨日为止的第4高“日峰”阈值
    labs_4d = thr4d.join(day_max_actual, how="left")
    labs_4d = labs_4d[labs_4d["is_summer_month_day"].astype(bool)]
    y4d = (labs_4d["day_max_actual"].values > labs_4d["m4_daily_prev"].values).astype(int)
    p4d = prob_all.reindex(labs_4d.index)["p_4cp_daily_monthlyTop4"].values

    # 7) 评估（固定阈值 0.5）
    m1 = _eval_binary(y1, p1, thr=0.5)
    m5 = _eval_binary(y5, p5, thr=0.5)
    m4h = _eval_binary(y4h, p4h, thr=0.5)
    m4d = _eval_binary(y4d, p4d, thr=0.5)

    # 8) 汇总与持久化
    probs_by_src[src] = prob_all.copy()
    # 把 4CP 标签一并存放，便于 Cell 10 校准曲线
    labels_by_src[src] = (labs.copy()
                          .join(labs_4h[["m4_hourly_prev"]], how="outer")
                          .join(labs_4d[["m4_daily_prev"]], how="outer"))

    summary_rows.append({
        "forecast_src": src,
        # 1CP hourly (summer)
        "1CP_hourly_brier": m1["brier"],
        "1CP_hourly_prec@0.5": m1["precision"],
        "1CP_hourly_rec@0.5": m1["recall"],
        "1CP_hourly_f1@0.5": m1["f1"],
        "1CP_hourly_auc": m1["auc"],
        # Top-5 hourly (summer)
        "Top5_hourly_brier": m5["brier"],
        "Top5_hourly_prec@0.5": m5["precision"],
        "Top5_hourly_rec@0.5": m5["recall"],
        "Top5_hourly_f1@0.5": m5["f1"],
        "Top5_hourly_auc": m5["auc"],
        # 4CP hourly (monthly Top-4)
        "4CP_hourly_brier": m4h["brier"],
        "4CP_hourly_prec@0.5": m4h["precision"],
        "4CP_hourly_rec@0.5": m4h["recall"],
        "4CP_hourly_f1@0.5": m4h["f1"],
        "4CP_hourly_auc": m4h["auc"],
        # 4CP daily  (monthly Top-4)
        "4CP_daily_brier": m4d["brier"],
        "4CP_daily_prec@0.5": m4d["precision"],
        "4CP_daily_rec@0.5": m4d["recall"],
        "4CP_daily_f1@0.5": m4d["f1"],
        "4CP_daily_auc": m4d["auc"],
        # band counts
        "cnt_RED_4CP_hourly": int((prob_all["sig_4cp_hourly_band"] == "RED").sum()),
        "cnt_ORANGE_4CP_hourly": int((prob_all["sig_4cp_hourly_band"] == "ORANGE").sum()),
        "cnt_YELLOW_4CP_hourly": int((prob_all["sig_4cp_hourly_band"] == "YELLOW").sum()),
        "cnt_GREEN_4CP_hourly": int((prob_all["sig_4cp_hourly_band"] == "GREEN").sum()),
        "cnt_RED_4CP_daily": int((prob_all["sig_4cp_daily_band"] == "RED").sum()),
        "cnt_ORANGE_4CP_daily": int((prob_all["sig_4cp_daily_band"] == "ORANGE").sum()),
        "cnt_YELLOW_4CP_daily": int((prob_all["sig_4cp_daily_band"] == "YELLOW").sum()),
        "cnt_GREEN_4CP_daily": int((prob_all["sig_4cp_daily_band"] == "GREEN").sum()),
    })

    if 'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS:
        out_csv = os.path.join(OUT_DIR, f"prob_hourly_pjm_{src}.csv")
        probs_by_src[src].to_csv(out_csv)
        print("Saved:", out_csv)

# 总结表
summary_df = pd.DataFrame(summary_rows).sort_values("forecast_src").reset_index(drop=True)
display(summary_df)
if 'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS:
    out_summary = os.path.join(OUT_DIR, "summary_pjm_hourly_cp_metrics.csv")
    summary_df.to_csv(out_summary, index=False)
    print("Saved:", out_summary)



>>> Running source: ECA — column: ECA: IESO-Ontario Demand Historic Forecast
Saved: ./output_cp_prob_mc_ieso\prob_hourly_pjm_ECA.csv

>>> Running source: RTO — column: RTO: IESO-Ontario Demand Historic Forecast
Saved: ./output_cp_prob_mc_ieso\prob_hourly_pjm_RTO.csv

>>> Running source: TESLA — column: TESLA: IESO-Ontario Demand Historic Forecast
Saved: ./output_cp_prob_mc_ieso\prob_hourly_pjm_TESLA.csv


,forecast_src,1CP_hourly_brier,1CP_hourly_prec@0.5,1CP_hourly_rec@0.5,1CP_hourly_f1@0.5,1CP_hourly_auc,Top5_hourly_brier,Top5_hourly_prec@0.5,Top5_hourly_rec@0.5,Top5_hourly_f1@0.5,Top5_hourly_auc,4CP_hourly_brier,4CP_hourly_prec@0.5,4CP_hourly_rec@0.5,4CP_hourly_f1@0.5,4CP_hourly_auc,4CP_daily_brier,4CP_daily_prec@0.5,4CP_daily_rec@0.5,4CP_daily_f1@0.5,4CP_daily_auc,cnt_RED_4CP_hourly,cnt_ORANGE_4CP_hourly,cnt_YELLOW_4CP_hourly,cnt_GREEN_4CP_hourly,cnt_RED_4CP_daily,cnt_ORANGE_4CP_daily,cnt_YELLOW_4CP_daily,cnt_GREEN_4CP_daily
0,ECA,0.027098,1.000000,0.111111,0.200000,0.949690,0.030127,1.000000,0.166667,0.285714,0.959601,0.077302,0.916667,0.314286,0.468085,0.926581,0.122097,0.714286,0.471698,0.568182,0.853151,9,2,3,222,17,11,19,165
1,RTO,0.009783,0.666667,0.888889,0.761905,0.997616,0.018470,0.733333,0.916667,0.814815,0.992391,0.037999,0.815789,0.885714,0.849315,0.985288,0.055429,0.854545,0.886792,0.870370,0.974605,32,4,5,195,45,8,4,155
2,TESLA,0.009783,0.666667,0.888889,0.761905,0.997616,0.018470,0.733333,0.916667,0.814815,0.992391,0.037999,0.815789,0.885714,0.849315,0.985288,0.055429,0.854545,0.886792,0.870370,0.974605,32,4,5,195,45,8,4,155


Saved: ./output_cp_prob_mc_ieso\summary_pjm_hourly_cp_metrics.csv


## 6. Visualization

In [13]:
# ==== Cell 10: Visualization (PJM summer, hourly CP, traffic-light bands) ====
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


# ---------- helpers ----------

def _band_color(band: str) -> str:
    return {
        "RED": "#d62728",
        "ORANGE": "#ff7f0e",
        "YELLOW": "#bcbd22",
        "GREEN": "#2ca02c"
    }.get(str(band), "#7f7f7f")


def _ensure_dir(path: str):
    d = os.path.dirname(path)
    if d and not os.path.exists(d):
        os.makedirs(d, exist_ok=True)


def plot_prob_timeseries(prob_df: pd.DataFrame, p_col: str, title: str,
                         save_path: str | None = None, show: bool = True):
    """
    Timeseries of probability with traffic-light shading and 0.5 decision line.
    Supports 1CP / Top-5 / 4CP (hourly & daily).
    """
    if prob_df.empty or p_col not in prob_df.columns:
        print(f"[WARN] plot_prob_timeseries: missing column {p_col}")
        return

    # --- 明确映射：概率列 -> 分级列（包含 4CP） ---
    BAND_FOR = {
        "p_1cp_hourly_summer": "sig_1cp_band",
        "p_top5_hourly_summer": "sig_top5_band",
        "p_4cp_hourly_monthlyTop4": "sig_4cp_hourly_band",
        "p_4cp_daily_monthlyTop4": "sig_4cp_daily_band",
    }
    band_col = BAND_FOR.get(p_col, None)

    # --- 连续日索引，保留 NaN（让非夏季段断开，不画“斜坡”） ---
    idx_min = prob_df.index.min()
    idx_max = prob_df.index.max()
    if pd.isna(idx_min) or pd.isna(idx_max):
        print(f"[WARN] plot_prob_timeseries: invalid index for {p_col}")
        return
    idx = pd.date_range(idx_min, idx_max, freq="D")
    p = prob_df[p_col].reindex(idx)  # 不 dropna，NaN 将打断折线

    fig, ax = plt.subplots(figsize=(14, 4))

    # 背景色带
    ax.axhspan(0.80, 1.00, alpha=0.08, color=_band_color("RED"))
    ax.axhspan(0.60, 0.80, alpha=0.06, color=_band_color("ORANGE"))
    ax.axhspan(0.40, 0.60, alpha=0.05, color=_band_color("YELLOW"))
    ax.axhspan(0.00, 0.40, alpha=0.04, color=_band_color("GREEN"))

    # 概率曲线（NaN 自动断开）
    ax.plot(p.index, p.values, lw=1.5, label=p_col)

    # 0.5 决策线
    ax.axhline(0.5, ls="--", lw=1, color="#444444", alpha=0.6, label="threshold=0.5")

    # 彩色散点（按分级列）
    if band_col and band_col in prob_df.columns:
        bands = prob_df[band_col].reindex(idx)
        finite_mask = np.isfinite(p.values)
        for b in ["RED", "ORANGE", "YELLOW", "GREEN"]:
            mask = (bands == b) & finite_mask
            if mask.any():
                ax.scatter(p.index[mask], p.values[mask], s=18,
                           color=_band_color(b), label=b, zorder=3, alpha=0.9)

    ax.set_ylim(0, 1.02)
    ax.yaxis.set_major_formatter(PercentFormatter(1.0))
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel("Probability")
    ax.grid(True, ls=":", alpha=0.35)
    # 避免重复图例项
    handles, labels = ax.get_legend_handles_labels()
    uniq = dict(zip(labels, handles))
    ax.legend(uniq.values(), uniq.keys(), ncol=5, fontsize=9, frameon=False)

    plt.tight_layout()
    if save_path:
        _ensure_dir(save_path)
        plt.savefig(save_path, dpi=140, bbox_inches="tight")
        plt.close(fig)
    elif show:
        plt.show()


def plot_band_counts(prob_df: pd.DataFrame, band_col: str, title: str,
                     save_path: str | None = None, show: bool = True):
    """
    Bar chart of traffic-light band counts.
    """
    if prob_df.empty or band_col not in prob_df.columns:
        print(f"[WARN] plot_band_counts: missing column {band_col}")
        return

    counts = prob_df[band_col].value_counts().reindex(["RED", "ORANGE", "YELLOW", "GREEN"]).fillna(0)
    fig, ax = plt.subplots(figsize=(8, 3.6))
    colors = [_band_color(b) for b in counts.index]
    ax.bar(counts.index, counts.values, color=colors)
    ax.set_title(title)
    ax.set_ylabel("Days (count)")
    ax.grid(axis="y", ls=":", alpha=0.35)
    for i, v in enumerate(counts.values):
        ax.text(i, v + max(counts.values) * 0.02 if counts.values.max() > 0 else 0.2, f"{int(v)}",
                ha="center", va="bottom", fontsize=10)
    plt.tight_layout()
    if save_path:
        _ensure_dir(save_path)
        plt.savefig(save_path, dpi=140, bbox_inches="tight")
        plt.close(fig)
    elif show:
        plt.show()


def calibration_curve(prob_df: pd.DataFrame,
                      labs_df: pd.DataFrame,
                      p_col: str,
                      label_kind: str,
                      n_bins: int = 10,
                      title: str = "",
                      save_path: str | None = None,
                      show: bool = True):
    """
    Reliability / calibration curve.
    Supported label_kind: '1cp', 'top5', '4cp_hourly', '4cp_daily'
    - 统一按夏季过滤：优先使用 labs_df['is_summer_day']，若无则使用 'is_summer_month_day'
    - 阈值列：
        1cp        -> 'summer_running_max_prev_hourly'
        top5       -> 'summer_top5_thr_prev_hourly'
        4cp_hourly -> 'm4_hourly_prev'
        4cp_daily  -> 'm4_daily_prev'
    """
    if prob_df.empty or p_col not in prob_df.columns:
        print(f"[WARN] calibration_curve: missing {p_col}")
        return

    labs = labs_df.copy()

    # 统一的夏季过滤：优先 is_summer_day（1CP/Top-5），否则 is_summer_month_day（4CP）
    summer_flag_col = None
    if "is_summer_day" in labs.columns:
        summer_flag_col = "is_summer_day"
    elif "is_summer_month_day" in labs.columns:
        summer_flag_col = "is_summer_month_day"

    if summer_flag_col is not None:
        labs = labs[labs[summer_flag_col].astype(bool)]

    # 选择阈值和构造标签
    if label_kind == "1cp":
        thr_col = "summer_running_max_prev_hourly"
        if thr_col not in labs.columns:
            print(f"[WARN] calibration_curve: missing column {thr_col}")
            return
        labs = labs[np.isfinite(labs[thr_col])]
        y = (labs["day_max_actual"].values > labs[thr_col].values).astype(int)

    elif label_kind == "top5":
        thr_col = "summer_top5_thr_prev_hourly"
        if thr_col not in labs.columns:
            print(f"[WARN] calibration_curve: missing column {thr_col}")
            return
        labs = labs[np.isfinite(labs[thr_col])]
        y = (labs["day_max_actual"].values > labs[thr_col].values).astype(int)

    elif label_kind == "4cp_hourly":
        thr_col = "m4_hourly_prev"
        if thr_col not in labs.columns:
            print(f"[WARN] calibration_curve: missing column {thr_col} for 4cp_hourly")
            return
        labs = labs[np.isfinite(labs[thr_col])]
        y = (labs["day_max_actual"].values > labs[thr_col].values).astype(int)

    elif label_kind == "4cp_daily":
        thr_col = "m4_daily_prev"
        if thr_col not in labs.columns:
            print(f"[WARN] calibration_curve: missing column {thr_col} for 4cp_daily")
            return
        labs = labs[np.isfinite(labs[thr_col])]
        y = (labs["day_max_actual"].values > labs[thr_col].values).astype(int)

    else:
        print(f"[WARN] calibration_curve: unknown label_kind={label_kind}")
        return

    # 与概率对齐（按日期索引）
    df = pd.DataFrame({
        "p": prob_df.reindex(labs.index)[p_col].values,
        "y": y
    }).dropna()
    if df.empty:
        print(f"[WARN] calibration_curve: no aligned data for {p_col} (label_kind={label_kind})")
        return

    # 分箱并绘图
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    df["bin"] = pd.cut(df["p"], bins=bins, include_lowest=True)
    agg = df.groupby("bin", observed=True).agg(p_mean=("p", "mean"), y_rate=("y", "mean"), n=("y", "size")).dropna()

    fig, ax = plt.subplots(figsize=(6.4, 6.0))
    ax.plot([0, 1], [0, 1], ls="--", color="#444444", alpha=0.6, label="Perfect")
    ax.plot(agg["p_mean"], agg["y_rate"], marker="o", lw=1.5, label="Observed")
    for xp, yp, nn in zip(agg["p_mean"], agg["y_rate"], agg["n"]):
        ax.text(xp, yp, f" n={nn}", fontsize=9, ha="left", va="bottom", alpha=0.75)

    ax.set_xlim(0, 1);
    ax.set_ylim(0, 1)
    ax.xaxis.set_major_formatter(PercentFormatter(1.0))
    ax.yaxis.set_major_formatter(PercentFormatter(1.0))
    ax.set_xlabel("Predicted probability")
    ax.set_ylabel("Observed frequency")
    ax.set_title(title if title else f"Calibration — {p_col} ({label_kind})")
    ax.grid(True, ls=":", alpha=0.35)
    ax.legend(frameon=False)
    plt.tight_layout()
    if save_path:
        _ensure_dir(save_path)
        plt.savefig(save_path, dpi=140, bbox_inches="tight")
        plt.close(fig)
    elif show:
        plt.show()


# ---------- per-source plots ----------
if not probs_by_src:
    print("[WARN] probs_by_src is empty — run Cell 9 first.")
else:
    for src, prob in probs_by_src.items():
        # 1) 原有：1CP / Top-5
        plot_prob_timeseries(
            prob_df=prob, p_col="p_1cp_hourly_summer",
            title=f"{src} — 1CP (Hourly, Summer) Probability",
            save_path=(os.path.join(OUT_DIR, f"ts_1cp_hourly_{src}.png") if (
                    'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
            show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))
        plot_prob_timeseries(
            prob_df=prob, p_col="p_top5_hourly_summer",
            title=f"{src} — Top-5 (Hourly, Summer) Probability",
            save_path=(os.path.join(OUT_DIR, f"ts_top5_hourly_{src}.png") if (
                    'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
            show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))

        # 2) 新增：4CP（小时口径 / 日口径）
        plot_prob_timeseries(
            prob_df=prob, p_col="p_4cp_hourly_monthlyTop4",
            title=f"{src} — 4CP (Hourly, Monthly Top-4, Jun–Sep) Probability",
            save_path=(os.path.join(OUT_DIR, f"ts_4cp_hourly_{src}.png") if (
                    'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
            show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))
        plot_prob_timeseries(
            prob_df=prob, p_col="p_4cp_daily_monthlyTop4",
            title=f"{src} — 4CP (Daily,  Monthly Top-4, Jun–Sep) Probability",
            save_path=(os.path.join(OUT_DIR, f"ts_4cp_daily_{src}.png") if (
                    'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
            show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))

        # 3) 分级计数柱状图
        plot_band_counts(
            prob_df=prob, band_col="sig_4cp_hourly_band",
            title=f"{src} — 4CP Hourly Traffic-Light Counts",
            save_path=(os.path.join(OUT_DIR, f"band_counts_4cp_hourly_{src}.png") if (
                    'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
            show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))
        plot_band_counts(
            prob_df=prob, band_col="sig_4cp_daily_band",
            title=f"{src} — 4CP Daily Traffic-Light Counts",
            save_path=(os.path.join(OUT_DIR, f"band_counts_4cp_daily_{src}.png") if (
                    'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
            show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))

        # 4) 校准曲线（需要 Cell 9 里把 4CP 标签列加入 labels_by_src）
        if src in labels_by_src:
            labs = labels_by_src[src]
            # 1CP / Top-5（原有）
            calibration_curve(
                prob_df=prob, labs_df=labs,
                p_col="p_1cp_hourly_summer", label_kind="1cp",
                n_bins=10,
                title=f"{src} — Calibration (1CP Hourly, Summer)",
                save_path=(os.path.join(OUT_DIR, f"calib_1cp_{src}.png") if (
                        'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
                show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))
            calibration_curve(
                prob_df=prob, labs_df=labs,
                p_col="p_top5_hourly_summer", label_kind="top5",
                n_bins=10,
                title=f"{src} — Calibration (Top-5 Hourly, Summer)",
                save_path=(os.path.join(OUT_DIR, f"calib_top5_{src}.png") if (
                        'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
                show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))
            # 4CP（新增）
            calibration_curve(
                prob_df=prob, labs_df=labs,
                p_col="p_4cp_hourly_monthlyTop4", label_kind="4cp_hourly",
                n_bins=10,
                title=f"{src} — Calibration (4CP Hourly, Monthly Top-4)",
                save_path=(os.path.join(OUT_DIR, f"calib_4cp_hourly_{src}.png") if (
                        'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
                show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))
            calibration_curve(
                prob_df=prob, labs_df=labs,
                p_col="p_4cp_daily_monthlyTop4", label_kind="4cp_daily",
                n_bins=10,
                title=f"{src} — Calibration (4CP Daily,  Monthly Top-4)",
                save_path=(os.path.join(OUT_DIR, f"calib_4cp_daily_{src}.png") if (
                        'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS) else None),
                show=('SAVE_OUTPUTS' in globals() and not SAVE_OUTPUTS))
        else:
            print(f"[INFO] No labels_by_src for {src}; skip 4CP calibration plots.")

# ---------- optional: show summary table again ----------
try:
    display(summary_df)
except Exception:
    pass

print("Cell 10 complete.")


,forecast_src,1CP_hourly_brier,1CP_hourly_prec@0.5,1CP_hourly_rec@0.5,1CP_hourly_f1@0.5,1CP_hourly_auc,Top5_hourly_brier,Top5_hourly_prec@0.5,Top5_hourly_rec@0.5,Top5_hourly_f1@0.5,Top5_hourly_auc,4CP_hourly_brier,4CP_hourly_prec@0.5,4CP_hourly_rec@0.5,4CP_hourly_f1@0.5,4CP_hourly_auc,4CP_daily_brier,4CP_daily_prec@0.5,4CP_daily_rec@0.5,4CP_daily_f1@0.5,4CP_daily_auc,cnt_RED_4CP_hourly,cnt_ORANGE_4CP_hourly,cnt_YELLOW_4CP_hourly,cnt_GREEN_4CP_hourly,cnt_RED_4CP_daily,cnt_ORANGE_4CP_daily,cnt_YELLOW_4CP_daily,cnt_GREEN_4CP_daily
0,ECA,0.027098,1.000000,0.111111,0.200000,0.949690,0.030127,1.000000,0.166667,0.285714,0.959601,0.077302,0.916667,0.314286,0.468085,0.926581,0.122097,0.714286,0.471698,0.568182,0.853151,9,2,3,222,17,11,19,165
1,RTO,0.009783,0.666667,0.888889,0.761905,0.997616,0.018470,0.733333,0.916667,0.814815,0.992391,0.037999,0.815789,0.885714,0.849315,0.985288,0.055429,0.854545,0.886792,0.870370,0.974605,32,4,5,195,45,8,4,155
2,TESLA,0.009783,0.666667,0.888889,0.761905,0.997616,0.018470,0.733333,0.916667,0.814815,0.992391,0.037999,0.815789,0.885714,0.849315,0.985288,0.055429,0.854545,0.886792,0.870370,0.974605,32,4,5,195,45,8,4,155


Cell 10 complete.


In [18]:
import pandas as pd
import numpy as np
import os


def _confusion_counts(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    TP = int(((y_true == 1) & (y_pred == 1)).sum())
    FP = int(((y_true == 0) & (y_pred == 1)).sum())
    TN = int(((y_true == 0) & (y_pred == 0)).sum())
    FN = int(((y_true == 1) & (y_pred == 0)).sum())
    return dict(TP=TP, FP=FP, TN=TN, FN=FN)


rows = []
missing_sources = []

needed_prob_cols = {
    "4cp_hourly": "p_4cp_hourly_monthlyTop4",
    "4cp_daily": "p_4cp_daily_monthlyTop4"
}

for src in FORECAST_SOURCES:
    if src not in probs_by_src or src not in labels_by_src:
        missing_sources.append(src)
        continue
    prob_df = probs_by_src[src]
    lab_df = labels_by_src[src]

    # day_max_actual
    if "day_max_actual" not in lab_df.columns:
        print(f"[WARN] {src}: 缺少 day_max_actual，跳过。")
        continue

    for kind, p_col in needed_prob_cols.items():
        thr_col = "m4_hourly_prev" if kind == "4cp_hourly" else "m4_daily_prev"
        if p_col not in prob_df.columns:
            print(f"[WARN] {src}: Missing probability column {p_col}, skip {kind}")
            continue
        if thr_col not in lab_df.columns:
            print(f"[WARN] {src}: Missing threshold column {thr_col}, skip {kind}")
            continue

        # 对齐索引
        idx = (prob_df.index
               .intersection(lab_df.index))
        # 过滤有效标签
        mask_valid = (
                lab_df.loc[idx, thr_col].replace([np.inf, -np.inf], np.nan).notna()
                & lab_df.loc[idx, "day_max_actual"].replace([np.inf, -np.inf], np.nan).notna()
                & prob_df.loc[idx, p_col].replace([np.inf, -np.inf], np.nan).notna()
        )
        idx_use = idx[mask_valid]

        if len(idx_use) == 0:
            print(f"[INFO] {src}: {kind} No valid sample.")
            continue

        y_true = (lab_df.loc[idx_use, "day_max_actual"].values >
                  lab_df.loc[idx_use, thr_col].values).astype(int)
        y_pred = (prob_df.loc[idx_use, p_col].values >= 0.5).astype(int)

        cm = _confusion_counts(y_true, y_pred)
        cm.update({
            "forecast_src": src,
            "kind": kind,
            "support": int(len(idx_use)),
            "pred_pos": int(y_pred.sum()),
            "actual_pos": int(y_true.sum())
        })
        rows.append(cm)

confmat_df = pd.DataFrame(rows).sort_values(["forecast_src", "kind"]).reset_index(drop=True)

print("4CP (hourly & daily) Confusion:")
display(confmat_df)

if 'SAVE_OUTPUTS' in globals() and SAVE_OUTPUTS:
    out_path = os.path.join(OUT_DIR, "confusion_4cp_hourly_daily.csv")
    confmat_df.to_csv(out_path, index=False)
    print("Saved:", out_path)

if missing_sources:
    print("The following sources lack probability or labels and are not involved in statistics:",
          ", ".join(missing_sources))

4CP (hourly & daily) Confusion:


,TP,FP,TN,FN,forecast_src,kind,support,pred_pos,actual_pos
0,25,10,149,28,ECA,4cp_daily,212,35,53
1,11,1,200,24,ECA,4cp_hourly,236,12,35
2,47,8,151,6,RTO,4cp_daily,212,55,53
3,31,7,194,4,RTO,4cp_hourly,236,38,35
4,47,8,151,6,TESLA,4cp_daily,212,55,53
5,31,7,194,4,TESLA,4cp_hourly,236,38,35


Saved: ./output_cp_prob_mc_ieso\confusion_4cp_hourly_daily.csv
